In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ESTCampAI/Deeplearning

/content/drive/MyDrive/ESTCampAI/Deeplearning


- 의존성 라이브러리 설치

In [3]:
# LangChain 기본 구조 및 핵심 모듈
!pip install -qU langchain langchain-core langchain-community langchain-openai

# langchain-chroma, faiss-cpu : 벡터 DB를 오픈소스로 제공
!pip install -q langchain-chroma faiss-cpu

# langchain-teddynote   : 테디노트(유튜버)가 만든 랭체인 확장 패키지
# !pip install -qU langchain-teddynote
# 25.11.25 수업 중 다수가 설치 안 됨 → 서버측 에러인듯?

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

- API 키 등록

In [ ]:
api_key = ""

# 환경변수 등록
import os

# 추가 - 지정하고 대입
os.environ["OPENAI_API_KEY"] = api_key

### `LC01_LangChain 기초, LLM 연동....ipynb`에서 수행한 건 **<U>Zero-Shot Prompting**</U>
- MessagePromptTemplate (역할부여) 제외


## 제로샷 프롬프팅 (Zero-Shot Prompting)
- Zero-Shot : AI 모델이 사전 학습 없이 새로운 작업이나 상황에 대응하는 방식
- Zero-Shot Prompting : **사전 예시 없이**, 단순히 요청만 주어져 모델이 답변을 생성하는 방식

> **특징**  
> - **<font color=red>추가 예시 없이</font>** 문제 설명만으로 작업을 수행함  
> - 모델은 오로지 프롬프트에 포함된 정보만을 기반으로 응답을 생성함  
> - **장점** : `빠르고 간단하게 적용할 수 있음`  
> - **단점** : `복잡한 문제나 구체적인 형식을 요구하는 작업에서는 모델의 응답이 기대에 못 미칠 수 있음`

### Zero-Shot Prompting 개선 방법
- Few-Shot Prompting
- 메모리 기능 사용
- 체인 결합 (여러개의 언어 모델 연결)
- RAG (검색, 증강, 생성)
- TAG (테이블(DB), 증강, 생성)
- Agent 등



## 퓨샷 프롬프팅 (Few-Shot Prompting)
- 몇 가지 예시(사례)를 프롬프트에 포함시켜, 모델이 원하는 출력 형식이나 문제 해결 방식을 학습하도록 돕는 방식

> **특징**  
> - <b><font color=red>예시를 제공</font></b>하여 모델에게 어떤 식으로 답변해야 하는지 안내함  
> - 예시를 통해 맥락을 명확하게 전달하고, 응답의 일관성과 정확도를 향상시킴  
> - **장점** : `모델이 복잡하거나 구체적인 작업을 보다 정확하게 수행할 수 있음`  
> - **단점** : `적절한 예시 선택이 중요하며, 프롬프트가 길어질 수 있음`

<br>

---

<br>

- 단순한 지시만으로는 모델이 의도된 작업을 충분히 이해하기 어려운 복잡하거나 미묘한 작업에 유용
- 모델 Finetuning이나 대규모 데이터셋이 실행 가능하지 않은 상황에서 LLM의 능력을 유연하고 효율적으로 활용하기 위한 강력한 도구

In [5]:
from langchain.chat_models import init_chat_model # 모델 초기화 클래스
from langchain_core.output_parsers import StrOutputParser # 텍스트만 반환해주는 출력파서

# 템플릿 정의하는 도구
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

In [6]:
# 예시 - 반대말 출력하는 간단한 예제
examples1 = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "sunny", "output": "rainy"},
    {"input": "surprised", "output": "calm"},
    {"input": "dry", "output": "humid"},
    {"input": "hot", "output": "cold"},
    {"input": "satisfied", "output": "dissatisfied"}
]

In [7]:
# 모델 초기화
llm = init_chat_model(model = "openai:gpt-4.1", temperature = 0, max_tokens = 500)

In [8]:
# 1. 1차 템플릿 객체 생성 → 형식(포멧)을 정의
# Input과 Output을 어떤 모양으로 LLM에게 보여줄지를 지정
prompt = PromptTemplate.from_template("Input : {input}\nOutput : {output}")

In [9]:
# 2. 최종 템플릿 객체 생성 → 예시, 형식(1차템플릿), 지시문를 무두 조합한 최종 프롬프트
fewshot_prompt = FewShotPromptTemplate(
    examples = examples1,    # LLM에게 보여줄 예시
    example_prompt = prompt, # 형식 지정
    prefix = "입력의 반대말을 출력한다.", # 예시 앞부분에 붙는 지시문 (규칙이나 맥락을 설명)
    suffix = "input : {input}",        # 예시 뒷부분에 붙는 지시문 (사용자 입력을 표현)
    input_variables = ["input"]        # 입력변수 명시 (suffix의 input에 들어갈 값)
)

In [10]:
# 최종 템플릿(퓨샷 템플릿) 테스트 - llm에게 전달할 완성된 프롬프트 확인용
print(fewshot_prompt.format(input = "여름"))

입력의 반대말을 출력한다.

Input : happy
Output : sad

Input : tall
Output : short

Input : sunny
Output : rainy

Input : surprised
Output : calm

Input : dry
Output : humid

Input : hot
Output : cold

Input : satisfied
Output : dissatisfied

input : 여름


In [11]:
# 체인 구성
chain = fewshot_prompt | llm | StrOutputParser()

In [12]:
# 체인 실행
print(chain.invoke("excited"))
print(chain.invoke("foggy"))
print(chain.invoke("어둡다"))

Output : bored
Output : clear
Output : 밝다


[ 실습 ] - 직접 퓨샷기반의 체인 구현해서 응답 받아보기

- "토끼는 풀을 좋아하고, 사자는 고기를 좋아한다."
- "토끼":"풀", "사자":"고기"

In [13]:
examples2 = [
    {
        "question":"철수는 빨강색을 좋아하고 영희는 파란색을 좋아한다.",
        "answer":'"철수":"빨강색", "영희":"파란색"'
    },
    {
        "question":"길동은 짜장을 좋아하고, 심청은 짬뽕을 좋아한다.",
        "answer":'"길동":"짜장","심청":"짬뽕"'
    },
    {
        "question":"맹구는 국어를 좋아하고 유리는 수학을 좋아한다.",
        "answer":'"맹구":"국어", "유리":"수학"'
    },
    {
        "question":"봉봉은 AI를 강의하고 균균은 사물인터넷을 강의한다.",
        "answer":'"봉봉":"AI","균균":"사물인터넷"'
    },
    {
        "question":"A는 서울에 살고, B는 제주도에 산다.",
        "answer":'"A":"서울", "B":"제주도"'
    }
]

In [14]:
prompt = PromptTemplate.from_template('"{question}"\n{answer}')
prefixes = ["각각의 주어, 목적어를 출력하라.", "주어, 목적어", "다음과 같이 출력", "알아서 출력", ""]
for prefix in prefixes:
  fewshot_prompt = FewShotPromptTemplate(
      examples = examples2,
      example_prompt = prompt,
      prefix = prefix,
      suffix = '"{question}"',
      input_variables = ["question"]
  )
  chain = fewshot_prompt | llm | StrOutputParser()
  print(f"\nprefix = {prefix}\n{chain.invoke("토끼는 풀을 좋아하고, 사자는 고기를 좋아한다.")}\n")
  print("=" * 20)


prefix = 각각의 주어, 목적어를 출력하라.
"토끼":"풀", "사자":"고기"


prefix = 주어, 목적어
"토끼는 풀을 좋아하고, 사자는 고기를 좋아한다."  
주어와 목적어를 추출하면 다음과 같습니다.

**"토끼":"풀", "사자":"고기"**

---

**설명:**  
- "토끼는 풀을 좋아하고" → 주어: 토끼, 목적어: 풀  
- "사자는 고기를 좋아한다" → 주어: 사자, 목적어: 고기


prefix = 다음과 같이 출력
아래와 같이 출력하시면 됩니다.

```
"철수는 빨강색을 좋아하고 영희는 파란색을 좋아한다."
"철수":"빨강색", "영희":"파란색"

"길동은 짜장을 좋아하고, 심청은 짬뽕을 좋아한다."
"길동":"짜장","심청":"짬뽕"

"맹구는 국어를 좋아하고 유리는 수학을 좋아한다."
"맹구":"국어", "유리":"수학"

"봉봉은 AI를 강의하고 균균은 사물인터넷을 강의한다."
"봉봉":"AI","균균":"사물인터넷"

"A는 서울에 살고, B는 제주도에 산다."
"A":"서울", "B":"제주도"

"토끼는 풀을 좋아하고, 사자는 고기를 좋아한다."
"토끼":"풀", "사자":"고기"
```

마지막 문장에 대한 출력도 아래와 같이 맞추었습니다.

**"토끼는 풀을 좋아하고, 사자는 고기를 좋아한다."**  
**"토끼":"풀", "사자":"고기"**


prefix = 알아서 출력
"토끼":"풀", "사자":"고기"


prefix = 
"토끼는 풀을 좋아하고, 사자는 고기를 좋아한다."  
이 문장을 예시와 같은 형식으로 정리하면 다음과 같습니다:

**"토끼":"풀", "사자":"고기"**



# Callbacks - 스트리밍(streaming)

- <font color=red>stream_response()</font> : 스트림 형태로 결과 출력
  - **<U>결괏값을 반환하지 않음**</U>
  - import하여 새롭게 초기화 하지 않는 경우, 2번씩 출력되는 문제 발생 `가끔 출력이 2번씩 나오는 문제가 존재함` (버전 업데이트 이전)
  - 다른 Callback 활용 가능

- 스트리밍 옵션은 <font color=red>질의에 대한 답변을 실시간</font>으로 받을 때 유용
- <font color=red>streaming=True</font>로 설정하고 스트리밍으로 답변을 받기 위한 <font color=red>StreamingStdOutCallbackHandler()</font>을 콜백으로 지정

> 오류가 뜨는 경우 : 라이브러리가 langchain.callbacks.streaming_stdout에서 langchain_core.callbacks.streaming_stdout로 변경됨

In [15]:
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [16]:
# LLM 초기화 - 콜백함수 등록
llm_model = init_chat_model(
    model = "openai:gpt-4.1", # 모델
    temperature = 0,          # 창의성 제어
    max_tokens = 500,         # 최대 출력토큰 개수
    streaming = True,         # 스트리밍 모드 활성화
    callbacks = [StreamingStdOutCallbackHandler()] # 콜백핸들러 적용 (결과값을 담으면서 출력하는 게 가능)
)

In [17]:
# 체인 구성
chain = llm_model | StrOutputParser()

# 실행
chain.invoke("RAG에 대해 500자 이내로 자세하게 설명해줘봐.")

RAG(Retrieval-Augmented Generation)은 자연어 처리(NLP)에서 정보 검색과 생성 모델을 결합한 기술입니다. 기존의 생성형 AI는 학습된 데이터만을 바탕으로 답변을 생성하지만, RAG는 외부 지식베이스나 문서에서 관련 정보를 먼저 검색한 후, 이를 바탕으로 답변을 생성합니다. 이 과정은 크게 두 단계로 나뉩니다. 첫째, 질문에 대해 관련 문서를 검색기(retriever)가 찾아냅니다. 둘째, 생성기(generator)가 검색된 문서의 내용을 참고하여 더 정확하고 신뢰성 있는 답변을 생성합니다. RAG는 최신 정보 반영, 사실성 향상, 출처 제공 등에서 강점을 가지며, 챗봇, 검색엔진, 지식관리 등 다양한 분야에 활용됩니다.

'RAG(Retrieval-Augmented Generation)은 자연어 처리(NLP)에서 정보 검색과 생성 모델을 결합한 기술입니다. 기존의 생성형 AI는 학습된 데이터만을 바탕으로 답변을 생성하지만, RAG는 외부 지식베이스나 문서에서 관련 정보를 먼저 검색한 후, 이를 바탕으로 답변을 생성합니다. 이 과정은 크게 두 단계로 나뉩니다. 첫째, 질문에 대해 관련 문서를 검색기(retriever)가 찾아냅니다. 둘째, 생성기(generator)가 검색된 문서의 내용을 참고하여 더 정확하고 신뢰성 있는 답변을 생성합니다. RAG는 최신 정보 반영, 사실성 향상, 출처 제공 등에서 강점을 가지며, 챗봇, 검색엔진, 지식관리 등 다양한 분야에 활용됩니다.'

[ 실습 ]

In [18]:
# Few‐Shot 예시: LangChain 주요 구성요소 설명
example3 = [
    {"input" : "Agent",           "output" : "프롬프트를 통해 도구를 호출하며 워크플로우를 제어할 수 있는 자동화 컴포넌트"},
    {"input" : "Chain",           "output" : "여러 프롬프트와 모델 호출을 순차적으로 연결해 파이프라인을 구성하는 구조"},
    {"input" : "PromptTemplate",  "output" : "입력 변수를 포함해 모델에 보낼 프롬프트 형식을 정의하는 클래스"}
]

In [19]:
# 1. 1차 템플릿 생성 - 형식 알려주기 용도
prompt = PromptTemplate.from_template("개념명 : {input}\n설명 : {output}")

In [20]:
# 2. 최종 템플릿 생성 - 최종 프롬프트 전달용
fewshot_prompt = FewShotPromptTemplate(
    examples = example3,
    example_prompt = prompt,
    prefix = "LangChain의 주요 구성요소를 다음 예시처럼 설명하세요.",
    suffix = "개념명 : {input}",
    input_variables = ["input"]
)

print(fewshot_prompt.format(input = "Tool"))

LangChain의 주요 구성요소를 다음 예시처럼 설명하세요.

개념명 : Agent
설명 : 프롬프트를 통해 도구를 호출하며 워크플로우를 제어할 수 있는 자동화 컴포넌트

개념명 : Chain
설명 : 여러 프롬프트와 모델 호출을 순차적으로 연결해 파이프라인을 구성하는 구조

개념명 : PromptTemplate
설명 : 입력 변수를 포함해 모델에 보낼 프롬프트 형식을 정의하는 클래스

개념명 : Tool


In [21]:
# 3. 체인 구성 - 콜백핸들러 세팅된 모델 적용
chain = fewshot_prompt | llm_model | StrOutputParser()

In [22]:
# 4. 체인 구성 - 콜백핸들러 세팅된 모델 적용
temp = chain.invoke("Tool")

개념명 : Tool  
설명 : 외부 API 호출, 계산, 데이터베이스 질의 등 특정 작업을 수행할 수 있도록 Agent가 사용할 수 있게 제공하는 기능 단위 컴포넌트

# ExampleSelector
- 예시가 많으면 토큰 사용량이 많아지고 따라서 API 사용료가 많이 나옴
- 최종 질문과 유사한 예시 한 두 개만 프롬프트에 포함할 예제를 선택하는 경우에 사용

- 종류
  - <font color=red>SemanticSimilarityExampleSelector</font>: 벡터 임베딩의 코사인 유사도를 이용해 질문과 의미가 가까운 예시를 선택
---
- 상황에 따라 사용
  - <font color=red>BaseExampleSelector</font>: 상속 받아 사용자 정의 ExampleSelector를 생성
  - <font color=red>LengthBasedExampleSelector</font>: 지정 길이가 넘어가지 않도록 예시 개수를 조절
  - <font color=red>MaxMarginalRelevanceExampleSelector</font>: Maximal Marginal Relevance (MMR)을 이용해 질문과 가까우면서도 다양한 예시를 선택
  - <font color=red>NGramOverlapExampleSelector</font>: n-gram overlap score를 이용해 질문과 가까운 예시를 선택

- 사용자 입력과의 유사도에 따라 예시를 선택해서 LLM 모델에 Few-show Prompting 하기 (질문에 대해 예시와 같이 답변하도록 하는 기능 구현)
  - 예시와 사용자 입력을 Text Embeddings 변환하여, Cosine Similarity가 가장 높은 k개의 예시를 선별해서 Few-shot Prompt에 넣어서 LLM 모델에 전달하는 방법

In [23]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector # 예시선택기
from langchain_openai import OpenAIEmbeddings # 임베딩 클래스
from langchain_community.vectorstores import FAISS

### 📌 Chroma vs FAISS 비교표

| 구분               | **Chroma**                 | **FAISS**                               |
| ---------------- | -------------------------- | --------------------------------------- |
| **설명**           | 로컬·파일 기반으로 쉽게 쓰는 가벼운 벡터 DB | 메타(Facebook)가 개발한 고성능 벡터 검색 엔진          |
| **설치/사용 난이도**    | ⭐ 매우 쉬움                    | ⭐⭐ 중간(조금 더 기술적)                         |
| **저장 방식**        | SQLite/Local 파일 기반         | 메모리/인덱스 파일 기반                           |
| **검색 속도**        | 중간                         | 매우 빠름(고성능)                              |
| **대규모 데이터**      | 중·소규모 데이터에 적합              | 대량(수십~수백만 벡터)에서도 고속 검색 가능               |
| **데이터 지속성**      | 파일로 저장 → 재사용 가능            | 인메모리 중심, 직접 저장을 구현해야 함(저장/로드 지원은 있음)    |
| **필요한 구성 요소**    | 파이썬만 있으면 동작(가볍고 간단)        | CPU 또는 GPU 환경 필요(특히 GPU 버전 매우 빠름)       |
| **문서화 & 예제**     | 교육용·입문용으로 설명이 쉽고 예제가 많음    | ML/IR(Information Retrieval) 영역에서 많이 쓰임 |
| **LangChain 연동** | `langchain-chroma`로 간단     | `FAISS.from_documents()`로 쉽게 연동 가능      |
| **장점**           | - 설치·사용이 매우 간단             |                                         |


In [24]:
# 모델 초기화

In [25]:
# 1) 긴 회의록 문장을 주면 → 정리된 회의록 형식으로 출력
# 2) 보고서 설명을 주면 → 핵심 요약을 출력

examples4 = [
    {
        "input": (
            "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. "
            "회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. "
            "회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. "
            "팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다."
        ),
        "answer": (
            "회의록: XYZ 회사 마케팅 전략 회의\n"
            "일시: 2023년 12월 25일\n"
            "장소: XYZ 회사 회의실\n"
            "참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n"
            "1. 개회\n"
            "   - 김수진 팀장의 개회사\n"
            "2. 시장 동향 개요 (김수진)\n"
            "   - 최근 시장 동향 분석\n"
            "3. 디지털 마케팅 전략 (박지민)\n"
            "   - SEO 최적화 및 온라인 광고 방안\n"
            "4. 소셜 미디어 캠페인 (이준호)\n"
            "   - 인플루언서 마케팅 제안\n"
            "5. 종합 논의\n"
            "   - 예산 및 자원 배분\n"
            "6. 마무리\n"
            "   - 다음 회의 일정 및 회의록 배포 담당자 지정"
        )
    },
    {
        "input": (
            "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. "
            "보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 지속 가능한 개발 전략을 다루고 있습니다. "
            "여러 국가의 성공 사례와 얻은 교훈도 포함되어 있습니다."
        ),
        "answer": (
            "문서 요약: 지속 가능한 도시 개발 전략 보고서\n\n"
            "- 중요성: 사회적·경제적·환경적 이점 강조\n"
            "- 문제점: 환경 오염, 자원 고갈, 불평등 분석\n"
            "- 전략: 친환경 건축, 대중교통 개선, 에너지 효율성 증대\n"
            "- 사례 연구: 코펜하겐, 요코하마 성공 사례\n"
            "- 교훈: 다각적 접근, 지역사회 협력, 장기 계획 필요"
        )
    }
]

In [26]:
# 1. 예시 선택기 생성
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples4,          # 예시 지정
    OpenAIEmbeddings(), # 임베딩 모델 (텍스트를 벡터로 변환)
    FAISS,              # 벡터 DB
    k = 1               # 입력과 가장 유사한 예시 1개를 선택
)

In [27]:
# 2. 1차 템플릿 객체 생성 - 형식 알려주기 용도
prompt = PromptTemplate.from_template(
    """
    문서 : {input}

    결과 : {answer}
    """
)

In [28]:
# 3. 최종 템플릿 객체 생성 - 최종 프롬프트를 완성시키는 용도
fewshot_prompt = FewShotPromptTemplate(
    example_selector = example_selector, # 예시 선택기
    example_prompt = prompt,             # 형식 알려주기
    prefix = "아래 예시를 참고해서, 입력된 텍스트에 대해 적절한 형식으로 회의록 또는 요약문을 작성해주세요.",
    suffix = "문서 : {input}",
    input_variables = ["input"]
)

In [29]:
# 4. 체인 구성
chain = fewshot_prompt | llm_model | StrOutputParser()

In [30]:
# 회의 메모 → 회의록 체인 실행
rs1 = chain.invoke({
    "input": (
        "2025년 5월 20일 오후 4시부터 약 90분간, 서울 본사 3층 대회의실에서 "
        "신제품 출시 킥오프 회의가 진행되었습니다. 회의에는 개발팀장 김민준, "
        "디자인팀장 이지은, 영업팀장 박성호, 마케팅팀장 오세훈, 품질관리 담당 최유리가 참석했으며, "
        "주요 안건은 제품 디자인 확정, 출시 일정, 마케팅 채널 전략, 리스크 관리 방안이었습니다."
    )
})

회의록: 신제품 출시 킥오프 회의  
일시: 2025년 5월 20일 오후 4시~5시 30분  
장소: 서울 본사 3층 대회의실  
참석자: 김민준(개발팀장), 이지은(디자인팀장), 박성호(영업팀장), 오세훈(마케팅팀장), 최유리(품질관리 담당)

1. 개회
   - 회의 시작 및 참석자 소개

2. 제품 디자인 확정 (이지은)
   - 최종 디자인 시안 발표 및 의견 수렴
   - 디자인 관련 주요 변경사항 공유

3. 출시 일정 논의 (김민준)
   - 개발 및 생산 일정 보고
   - 출시 목표일 확정

4. 마케팅 채널 전략 (오세훈)
   - 온·오프라인 마케팅 채널 제안
   - 초기 프로모션 방안 논의

5. 리스크 관리 방안 (최유리)
   - 품질관리 체크리스트 공유
   - 예상 리스크 및 대응 방안 논의

6. 종합 논의
   - 부서별 협업 방안 및 일정 조율

7. 마무리
   - 다음 회의 일정 안내
   - 회의록 작성 및 배포 담당자 지정

In [31]:
# 보고서 → 보고서 요약 체인 실행
rs2 = chain.invoke({
    "input": (
        "이 문서는 '머신러닝 모델 성능 최적화 기법'에 대한 15페이지 분량의 기술 보고서입니다. "
        "보고서는 하이퍼파라미터 튜닝(그리드 서치, 베이지안 최적화), 데이터 증강(회전·크롭·색상 보정), "
        "앙상블 기법(배깅·부스팅·스태킹), 모델 경량화(지식 증류·양자화), 성능 평가(Accuracy·Precision·Recall·F1) 등을 상세히 다룹니다."
    )
})

문서 요약: 머신러닝 모델 성능 최적화 기법 보고서

- 하이퍼파라미터 튜닝: 그리드 서치, 베이지안 최적화 등 다양한 방법 소개
- 데이터 증강: 회전, 크롭, 색상 보정 등으로 데이터 다양성 확보
- 앙상블 기법: 배깅, 부스팅, 스태킹을 통한 성능 향상
- 모델 경량화: 지식 증류, 양자화로 모델 효율성 증대
- 성능 평가: Accuracy, Precision, Recall, F1 등 다양한 지표 활용

# MessagePromptTemplate 활용 - 역할을 부여하는 것
- <font color=red>AI 메시지, 시스템 메시지, 사용자 메시지를 사용</font>하여 적절한 맥락을 유지하고, 사용자와의 상호작용을 보다 자연스럽게 처리
  - <font color=red>SystemMessagePromptTemplate</font> : 시스템 프롬프트 설정
  - <font color=red>HumanMessagePromptTemplate</font> : 사용자 프롬프트 설정
- <font color=red>ChatPromptTemplate.from_messages()</font> : 시스템 메시지와 사용자 메시지 템플릿을 포함하는 챗 프롬프트를 구성
- <font color=red>chat_prompt.format_messages()</font> : 사용자의 질문을 포함한 메시지 리스트를 동적으로 생성하여 반환

- 생성된 메시지 리스트는 대화형 인터페이스나 언어 모델과의 상호작용을 위한 입력으로 사용
- 각 메시지는 <font color=red>role</font> (메시지를 말하는 주체, 여기서는 system 또는 user)과 <font color=red>content</font> (메시지의 내용) 속성을 포함
- 이 구조는 시스템과 사용자 간의 대화 흐름을 명확하게 표현하며, 언어 모델이 이를 기반으로 적절한 응답을 생성할 수 있도록 지원

- 또 다른 사용법

```python
# 시스템 메시지 정의
system_message = SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다")

# 인간 사용자 메시지 정의
human_message = HumanMessagePromptTemplate.from_template("{user_input}")

# 대화형 템플릿 생성
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
```

In [32]:
from langchain_core.prompts import ChatPromptTemplate

In [33]:
# 모델 초기화 - llm_model 활용
messages = [
    ("system", "너는 천문학 전문가이고, 천문학 관련 질문에 전문가 수준으로 대답할 수 있다."),
    ("human", "{user_input}")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [34]:
# llm에게 전달 될 최종 메세지 프롬프트 출력 (확인 용도)
prompt.format_messages(user_input = "태양계에서 가장 큰 행성은 무엇인가요?")

[SystemMessage(content='너는 천문학 전문가이고, 천문학 관련 질문에 전문가 수준으로 대답할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [35]:
# 체인 구성
chain = prompt | llm_model | StrOutputParser()

# 체인 실행
response = chain.invoke("태양계에서 가장 큰 행성은 무엇인가요?")

태양계에서 가장 큰 행성은 목성(Jupiter)입니다.  
목성은 지름이 약 142,984km로, 지구의 약 11배에 달하며, 질량은 지구의 약 318배에 이릅니다.  
목성은 주로 수소와 헬륨으로 이루어진 거대한 가스 행성으로, 태양계의 모든 다른 행성의 질량을 합친 것보다도 더 무겁습니다.  
또한, 목성은 강력한 자기장과 수많은 위성(대표적으로 이오, 유로파, 가니메데, 칼리스토 등)을 가지고 있습니다.

# FewShotChatMessagePromptTemplate
- <font color=red>Example Selector를 설정해주고, ChatPromptTemplate.from_messages()에 "human"과 "ai"의 메시지 포맷을 설정</font>
- 입력으로 받는 변수 input_variables=["input"] 을 설정

- Example Selector 의 유사도 검색 문제 해결
  - 유사도 계산시 instruction 과 input 을 사용하고 있지만, instruction 만 사용하여 검색시 제대로된 유사도 결과가 나오지 않음
  - 이를 해결하기 위해 커스텀 유사도 계산을 위한 클래스를 정의하여 사용

In [36]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_community.vectorstores import Chroma # 벡터 DB

In [37]:
# 첫 번째 예시 : 회의 메모에 대한 메세지 → 회의록 형식으로 답변하는 예시
# 두 번째 예시 : 보고서에 대한 메세지 → 요약문으로 답변하는 예시
examples5 = [
    {
        "ex_instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "ex_input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "ex_answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
"""
    },
    {
        "ex_instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "ex_input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "ex_answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
"""
    }
]

In [38]:
# 모델 초기화 - llm_model 활용

In [41]:
# 1. 예시 선택기 생성
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples = examples5,
    embeddings = OpenAIEmbeddings(),
    vectorstore_cls = Chroma,
    k = 1
)

In [42]:
# 2. 메세지 템플릿 객체 생성 - 표현 양식 알려주는 용도

#
messages = [
    ("system", "{ex_instruction}"), # 역할이나 톤
    ("human", "{ex_input}"),        # 사용자 입력
    ("ai", "{ex_answer}")           # 출력 형식
]

example_prompt = ChatPromptTemplate.from_messages(messages)

In [43]:
# 3. 퓨샷 템플릿 객체 생성 - 예시선택기, 메세지 템플릿을 엮어주는 역할
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector = example_selector, # 예시 선택기
    example_prompt = example_prompt      # 표현 양식 지정
)

In [47]:
# 4. 최종 템플릿(메세지 템플릿) - 최종 프롬프트를 만들어주는 역할
# 사용자 입력 (invoke, RunnablePathThrough 등)에서의 dict 키와 매핑
input_messages = [
    ("system", "{instruction}"), # 역할, 톤 설정
    few_shot_prompt,              # 퓨샷 템플릿(예시 선택기+표현 양식) 세팅
    ("human", "{input}")         # 실제 사용자 입력
]

final_prompt = ChatPromptTemplate.from_messages(input_messages)

In [48]:
# 5. 체인 생성
chain = final_prompt | llm_model | StrOutputParser()

In [49]:
# 첫 번째 질의 - 회의 메모 → 화의록 작성
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": (
        "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 "
        "주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, "
        "UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, "
        "다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, "
        "팀은 다음 주까지의 목표를 설정했다."
    ),
}

In [50]:
chain.invoke(question)

회의록: ABC 기술 회사 제품 개발팀 주간 진행 상황 회의  
일시: 2023년 12월 26일  
장소: ABC 기술 회사 회의실  
참석자: 최현수(프로젝트 매니저), 황지연(주요 개발자), 김태영(UI/UX 디자이너)

1. 개회  
- 최현수 프로젝트 매니저가 회의를 시작하며 회의 목적(프로젝트 진행 상황 검토 및 마일스톤 계획 수립)을 안내함.

2. 각 팀원별 진행 상황 보고  
- 황지연(주요 개발자):  
  · 백엔드 서버 연동 및 주요 기능 구현 현황 보고  
  · 현재까지 발견된 주요 이슈 및 해결 방안 공유

- 김태영(UI/UX 디자이너):  
  · 메인 화면 및 사용자 흐름 설계 진행 상황 설명  
  · 사용자 피드백 반영 계획 및 디자인 수정 일정 안내

3. 마일스톤 계획 수립  
- 다가오는 마일스톤(1차 프로토타입 완성)에 대한 일정 재확인  
- 각 담당자별로 다음 주까지 완료해야 할 주요 작업 항목 정리  
  · 개발: 로그인/회원가입 기능 완성  
  · 디자인: 세부 UI 요소 디자인 및 프로토타입 반영

4. 종합 논의  
- 일정 내 주요 이슈 발생 시 신속한 공유 및 협업 방안 논의  
- 다음 회의 전까지 각자 진행 상황을 문서로 정리하여 공유하기로 함

5. 마무리  
- 다음 회의 일정: 2024년 1월 2일(화) 오후 2시  
- 회의록 작성 및 배포: 최현수 담당

회의 종료: 오후 3시 40분

'회의록: ABC 기술 회사 제품 개발팀 주간 진행 상황 회의  \n일시: 2023년 12월 26일  \n장소: ABC 기술 회사 회의실  \n참석자: 최현수(프로젝트 매니저), 황지연(주요 개발자), 김태영(UI/UX 디자이너)\n\n1. 개회  \n- 최현수 프로젝트 매니저가 회의를 시작하며 회의 목적(프로젝트 진행 상황 검토 및 마일스톤 계획 수립)을 안내함.\n\n2. 각 팀원별 진행 상황 보고  \n- 황지연(주요 개발자):  \n  · 백엔드 서버 연동 및 주요 기능 구현 현황 보고  \n  · 현재까지 발견된 주요 이슈 및 해결 방안 공유\n\n- 김태영(UI/UX 디자이너):  \n  · 메인 화면 및 사용자 흐름 설계 진행 상황 설명  \n  · 사용자 피드백 반영 계획 및 디자인 수정 일정 안내\n\n3. 마일스톤 계획 수립  \n- 다가오는 마일스톤(1차 프로토타입 완성)에 대한 일정 재확인  \n- 각 담당자별로 다음 주까지 완료해야 할 주요 작업 항목 정리  \n  · 개발: 로그인/회원가입 기능 완성  \n  · 디자인: 세부 UI 요소 디자인 및 프로토타입 반영\n\n4. 종합 논의  \n- 일정 내 주요 이슈 발생 시 신속한 공유 및 협업 방안 논의  \n- 다음 회의 전까지 각자 진행 상황을 문서로 정리하여 공유하기로 함\n\n5. 마무리  \n- 다음 회의 일정: 2024년 1월 2일(화) 오후 2시  \n- 회의록 작성 및 배포: 최현수 담당\n\n회의 종료: 오후 3시 40분'

# 여러 개의 프롬프트 연결하기
- PromptTemplate을 이용해서 <font color=red>+</font> 연산자를 이용해서 여러개의 템플릿을 연결
  - 문자열 + 문자열
  - PromptTemplate + PromptTemplate
  - PromptTemplate + 문자열

---
- 프롬프트 조립용

- [실습] 프롬프트 추가해보기

# 순차적인 체인 연결
- RunableParellel() 은 병렬 수행
- 첫 번째 체인 : 한국어 단어를 영어로 번역하는 작업을 수행
- 두 번째 체인 : 해당 단어를 설명하는 작업을 수행